In [1]:
import os
import sys
import pandas as pd
import re


In [2]:
test_df = pd.read_csv('../../data/german/test.csv')
train_df = pd.read_csv('../../data/german/train.csv')
val_df = pd.read_csv('../../data/german/valid.csv')


test_df.head()

,text,label,source
0,@user korrekt! Verstehe sowas nicht...,negative,sb_10k
1,Einparken können die Aliens auch nicht! #schlefaz,neutral,sb_10k
2,Der Dubbletimepart von Julien war ja mal sowas...,positive,sb_10k
3,#Instachallenge #Day16 #what #i #am #reading #...,negative,sb_10k
4,Also gleich. Mach noch das Video fertig.,neutral,sb_10k


In [3]:
# from sklearn.model_selection import train_test_split

# train_df, _ = train_test_split(
#     train_df,
#     test_size = 0.6 ,
#     stratify = train_df['label'],
#     random_state = 42
# )

In [4]:

mapping = {'negative': 0, 'neutral': 1, 'positive': 2}

# Create a new column with mapped values
train_df['sentiment_num'] = train_df['label'].map(mapping)
test_df['sentiment_num'] = test_df['label'].map(mapping)
val_df['sentiment_num'] = val_df['label'].map(mapping)

In [5]:
url_pattern = re.compile(r"http\S+|www\.\S+")
mention_pattern = re.compile(r"@\w+")
hashtag_pattern = re.compile(r"#(\w+)")
rt_pattern = re.compile(r"^RT\s+@\w+:\s*")

def normalize_elongation(word, max_repeat=2):
    return re.sub(r"(.)\1{"+str(max_repeat)+r",}", r"\1"*max_repeat, word)

def clean_tweet(text: str) -> str:
    if not isinstance(text, str):
        return ""
    
    text = rt_pattern.sub("", text)
    text = url_pattern.sub(" URL ", text)
    text = mention_pattern.sub(" @usuario ", text)
    text = hashtag_pattern.sub(r"\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.lower()
    tokens = []
    for tok in text.split():
        tokens.append(normalize_elongation(tok))
    text = " ".join(tokens)

    return text



train_df["clean_text"] = train_df["text"].apply(clean_tweet)
test_df["clean_text"] = test_df["text"].apply(clean_tweet)
val_df["clean_text"] = val_df["text"].apply(clean_tweet)

In [ ]:
import torch

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW, XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification
from datasets import Dataset

model_name = "xlm-roberta-base"
tok = XLMRobertaTokenizerFast.from_pretrained(model_name)

MAX_LEN = 128 
def tokenize(batch):
    return tok(
        batch["text"].tolist(),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_encodings = tokenize(train_df)
val_encodings = tokenize(val_df)
test_encodings = tokenize(test_df)


/opt/miniconda3/envs/mbertproj/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/mbertproj/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SentimentDataset(train_encodings, train_df["sentiment_num"].values)
val_dataset = SentimentDataset(val_encodings, val_df["sentiment_num"].values)
test_dataset = SentimentDataset(test_encodings, test_df["sentiment_num"].values)

In [8]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, shuffle = True, batch_size = 8)
#change number of labels based on keep/drop neutral class

label2id = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

id2label = {v: k for k, v in label2id.items()}

num_labels = len(label2id)
model = XLMRobertaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
)
training_args = TrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/miniconda3/envs/mbertproj/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 20%|██        | 115/575 [01:29<06:39,  1.15it/s]

{'loss': 1.0165, 'grad_norm': 13.730364799499512, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.0}


                                                 
 20%|██        | 115/575 [01:33<06:39,  1.15it/s]

{'eval_loss': 0.8808032274246216, 'eval_runtime': 3.4314, 'eval_samples_per_second': 94.423, 'eval_steps_per_second': 6.12, 'epoch': 1.0}


 40%|████      | 230/575 [03:07<04:40,  1.23it/s]

...

                                                 
100%|██████████| 575/575 [08:57<00:00,  1.08it/s]

{'eval_loss': 0.7033093571662903, 'eval_runtime': 3.1146, 'eval_samples_per_second': 104.028, 'eval_steps_per_second': 6.743, 'epoch': 5.0}


100%|██████████| 575/575 [09:01<00:00,  1.06it/s]

{'train_runtime': 541.1271, 'train_samples_per_second': 16.992, 'train_steps_per_second': 1.063, 'train_loss': 0.695913430918818, 'epoch': 5.0}


TrainOutput(global_step=575, training_loss=0.695913430918818, metrics={'train_runtime': 541.1271, 'train_samples_per_second': 16.992, 'train_steps_per_second': 1.063, 'total_flos': 472524975783000.0, 'train_loss': 0.695913430918818, 'epoch': 5.0})

In [9]:
metrics = trainer.evaluate()
print(metrics)

100%|██████████| 21/21 [00:03<00:00,  6.84it/s]

{'eval_loss': 0.7033093571662903, 'eval_runtime': 3.0868, 'eval_samples_per_second': 104.965, 'eval_steps_per_second': 6.803, 'epoch': 5.0}


In [10]:
predictions = trainer.predict(val_dataset)

100%|██████████| 21/21 [00:03<00:00,  5.35it/s]


In [12]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, classification_report

logits = predictions.predictions      # or predictions[0]
labels = predictions.label_ids        # or predictions[1]

y_pred = np.argmax(logits, axis=-1)

print("Accuracy:", accuracy_score(labels, y_pred))
print("F1 (macro):", f1_score(labels, y_pred, average="macro"))

print(
    classification_report(
        labels,
        y_pred,
        target_names=["negative", "neutral", "positive"]
    )
)

Accuracy: 0.7438271604938271
F1 (macro): 0.7423535727288263
              precision    recall  f1-score   support

    negative       0.71      0.74      0.72       108
     neutral       0.75      0.66      0.70       108
    positive       0.78      0.83      0.80       108

    accuracy                           0.74       324
   macro avg       0.74      0.74      0.74       324
weighted avg       0.74      0.74      0.74       324



Test Dataset

In [13]:
pred = trainer.predict(test_dataset)
y_true = test_df["sentiment_num"]
y_pred = np.argmax(pred.predictions, axis=1)


logits = pred.predictions   
labels = pred.label_ids   


print("Accuracy:", accuracy_score(labels, y_pred))
print("F1 (macro):", f1_score(labels, y_pred, average="macro"))

print(
    classification_report(
        labels,
        y_pred,
        #target_names=["negative", "neutral", "positive"]
        target_names=["negative","neutral", "positive"]
    )
)

100%|██████████| 21/21 [00:02<00:00,  7.40it/s]


Accuracy: 0.7438271604938271
F1 (macro): 0.7436737945019444
              precision    recall  f1-score   support

    negative       0.70      0.75      0.73       108
     neutral       0.75      0.69      0.72       108
    positive       0.78      0.79      0.78       108

    accuracy                           0.74       324
   macro avg       0.74      0.74      0.74       324
weighted avg       0.74      0.74      0.74       324

